In [159]:
from nltk.corpus import wordnet as wn
import nltk 
import re

In [160]:
def remove_punctuation(sentence):
    return re.sub(r'[^\w\s]', '', sentence).strip()

def get_stopwords():
    stopwords_file = open(f"./resources/utils/stop_words__frakes_baeza-yates.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()

    stopwords_file = open(f"./resources/utils/stop_words_1.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()

    stopwords_file = open(f"./resources/utils/stop_words_FULL.txt", "r")
    stopwords_list = []
    for word in stopwords_file:
        stopwords_list.append(word.replace('\n', ''))
    stopwords_file.close()
    
    stopwords_list = list(set(stopwords_list))

    return stopwords_list
     
def remove_stopwords(sentence, stopwords_list):
    return [word for word in sentence.split() if word not in stopwords_list]



### Esercitazione su Word Sense Disambiguation: 

#### INPUT : 
l’input per questa esercitazione è costituito da coppie di termini contenute nel file WordSim353 (disponibile nei
formati .tsv e .csv)
- Il file contiene 353 coppie di termini utilizzati come testset in varie competizioni internazionali
- A ciascuna coppia è attribuito un valore numerico [0,10], che rappresenta la similarità fra gli elementi della coppia.

In [161]:
import csv

sim_file = f"resources/WordSim353/WordSim353.csv"

pairs = []
with open(sim_file, 'r', encoding = "utf-8") as f:
    reader = csv.reader(f)
    reader.__next__()
    for lines in reader:
        pairs.append(lines)
    
print("WordSim353 correctly loaded!")


WordSim353 correctly loaded!


#### Prima parte

Implementare tre misure di similarità basate su WordNet. Per ciascuna delle misure di similarità, calcolare:
- gli indici di correlazione di Spearman e
- gli indici di correlazione di Pearson fra i risultati ottenuti e quelli ‘target’ presenti nel file annotato.

Le misure da implementare sono le seguenti: 
- Wu & Palmer: 
$$cs(s_1, s_2) = \frac{2 \cdot depth(LCS)}{depth(s_1) + depth(s_2)}$$

in cui $LCS$ rappresenta il primo antenato comune (Lowest Common Subsumer) e depth è la funzione che misura la distanza fra la radice di WordNet e il sysnet $x$. 

***L’obiettivo è implementare la misura di similarità di Wu & Palmer! Non dobbiamo fare ciò viene già fatto nell’implementazione di nltk o altre librerie!***



In [162]:
def wu_palmer_similarity(synset1, synset2):
    """
    Returns the Wu-Palmer similarity between synset.
    """

    if isinstance(synset1, nltk.corpus.reader.wordnet.Synset) and isinstance(synset2, nltk.corpus.reader.wordnet.Synset):
        lcs = synset1.lowest_common_hypernyms(synset2) # QUESTO NON SI PUO' USARE
    else:
        raise TypeError("The input parameters must be Synset objects")

    

    




    return 0


"""

    oggetto

    veicolo


barca      automobile 


            macchina
"""

'\n\n    oggetto\n\n    veicolo\n\n\nbarca      automobile \n\n\n            macchina\n'

- Shortest Path:
$$sim_{path}(s_1, s_2) = 2 \cdot depthMax - len(s_1, s_2)$$

in cui $depthMax$ è un valore fissato per una specifica versione di WordNet. 

- Leakcock & Chodorow: 
$$sim_{LC}(s_1, s_2) = - log \frac{len(s_1, s_2)}{2 \cdot depthMax}$$

#### Calcolo della similarità

Le funzioni precedentemente implementate richiedono come input sensi e non i termini. Quindi per calcolare la similarità fra 2 termini è necessario prendere la massima similarità fra tutti i sensi del primo termine e tutti i sensi del secondo termine. 

L'ipotesi quindi è che i due termini funzionino come contesto di disambiguazione l'uno per l'altro. 

L'equazione che formalizza questa idea è la seguente: 

$$sim(w_1, w_2) = \max_{c_1 \in s(w_1), c_2 \in s(w_2)} [sim(c_1, c_2)]$$

***NB!*** Per calcolare gli indici di correlazione non si è interessati a entrare nel merito di come sono calcolati, possiamo prendere delle funzioni prefatte e usarle.

### Seconda parte

Implementare l’algoritmo di Lesk (NON (!=) usare implementazione esistente, e.g., in nltk…).
1. Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di
WN) e disambiguare (almeno) un sostantivo per frase. 
Calcolare l’accuratezza del sistema implementato sulla base dei sensi annotati in
SemCor.
    - SemCor è disponibile all’URL http://web.eecs.umich.edu/~mihalcea/downloads.html
2. Randomizzare la selezione delle 50 frasi e la selezione del termine da disambiguare, 
e restituire l’accuratezza media su (per esempio) 10 esecuzioni del programma.

In [163]:
stop_words = get_stopwords()

In [164]:
def get_set_of_words(phrase):
    """
    Returns the set of words of a phrase.
    """
    phrase = remove_punctuation(phrase)
    set_of_words = remove_stopwords(phrase, stop_words)
    
    return set_of_words

In [165]:
def get_context_of_phrase(sentence):
    """
    Returns the context of a phrase.
    """
    sentence = remove_punctuation(sentence)
    set_of_words = remove_stopwords(sentence, stop_words)
    return set_of_words

def compute_overlap(signature, context):
    """
    returns the number of words in common between signature and context
    """

    number_of_words_in_common = len(list(set(signature) & set(context)))

    return number_of_words_in_common

def get_signature(synset):
    """
    returns the signature of synset
    """

    gloss_of_synset = synset.definition() #gloss of synset
    examples_of_synset = synset.examples() #examples of synset

    # print("\ngloss_of_synset")
    # print(gloss_of_synset)

    # print("examples_of_synset")
    # print(examples_of_synset)

    initial_signature = [gloss_of_synset] + examples_of_synset

    signature = []
    for phrase in initial_signature:
        set_of_words = get_set_of_words(phrase)
        signature.append(set_of_words)

    result = sum(signature, []) # flatten list of lists
    return list(set(result)) # remove duplicates

def lesk(word, sentence):
    """
    returns best sense of word
    """

    best_sense = None #most frequent sense for word
    max_overlap = 0
    context = get_context_of_phrase(sentence) #set of words in sentence

    print("CONTEXT:")
    print(context)

    for synset in wn.synsets(word):
        print("\n" + str(synset))
        signature = get_signature(synset) #set of words in the gloss and examples of sense

        print("SIGNATURE:")
        print(signature)

        overlap = compute_overlap(signature, context)
        print("OVERLAP: " + str(overlap) + "\n")
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = synset

    return best_sense

In [166]:
best_sense = lesk("dog", "I saw a dog in the park")

print("best sense:")
print(best_sense) 

CONTEXT:
['I', 'dog', 'park']

Synset('dog.n.01')
SIGNATURE:
['member', 'genus', 'domesticated', 'common', 'barked', 'Canis', 'times', 'occurs', 'breeds', 'descended', 'wolf', 'dog', 'night', 'man', 'prehistoric']
OVERLAP: 1


Synset('frump.n.01')
SIGNATURE:
['girl', 'dull', 'frump', 'dog', 'unattractive', 'real', 'unpleasant', 'woman', 'reputation']
OVERLAP: 1


Synset('dog.n.03')
SIGNATURE:
['informal', 'term', 'lucky', 'dog', 'man']
OVERLAP: 1


Synset('cad.n.01')
SIGNATURE:
['dirty', 'reprehensible', 'morally', 'dog']
OVERLAP: 1


Synset('frank.n.02')
SIGNATURE:
['sausage', 'served', 'minced', 'smoothtextured', 'smoked', 'roll', 'bread', 'pork', 'beef']
OVERLAP: 0


Synset('pawl.n.01')
SIGNATURE:
['notch', 'ratchet', 'wheel', 'forward', 'hinged', 'backward', 'fits', 'catch', 'moving', 'move', 'prevent']
OVERLAP: 0


Synset('andiron.n.01')
SIGNATURE:
['fireplace', 'andirons', 'logs', 'touch', 'supports', 'hot', 'metal']
OVERLAP: 0


Synset('chase.v.01')
SIGNATURE:
['intent', 'dog', 